In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
import re
import subprocess
from dotenv import load_dotenv
import customtkinter as ctk


login_url = "https://anitaku.pe/login.html"
base_url = "https://anitaku.pe/category/one-piece"
serie = "One_Piece"

load_dotenv('.secrets')

email = os.getenv('EMAIL')
password = os.getenv('PASSWORD')
file_path = os.getenv('FILE_PATH')

quality = 4 
lastep = 5
idm = True # Sin idm, 5 episodios de máxima calidad tardan aproximadamente 15 minutos, con IDM tardan aproximadamente 3 minutos

In [2]:
# Inicializar la ventana principal
app = ctk.CTk()
app.title("Configuración de Variables")
app.geometry("400x400")

# Definir las variables
login_url = ctk.StringVar(value="https://anitaku.pe/login.html")
base_url = ctk.StringVar(value="https://anitaku.pe/category/one-piece")
serie = ctk.StringVar(value="One_Piece")
quality = ctk.IntVar(value=4)
lastep = ctk.IntVar(value=5)
idm = ctk.BooleanVar(value=True)

# Crear las etiquetas y entradas para cada variable
def create_label_entry(label_text, variable):
    frame = ctk.CTkFrame(app)
    frame.pack(pady=5, padx=10, fill="x")
    
    label = ctk.CTkLabel(frame, text=label_text)
    label.pack(side="left", padx=5)
    
    entry = ctk.CTkEntry(frame, textvariable=variable)
    entry.pack(side="right", fill="x", expand=True)
    return entry

create_label_entry("Login URL:", login_url)
create_label_entry("Base URL:", base_url)
create_label_entry("Serie:", serie)

# Crear las etiquetas y sliders para quality y lastep
def create_label_slider(label_text, variable, from_, to_):
    frame = ctk.CTkFrame(app)
    frame.pack(pady=5, padx=10, fill="x")
    
    label = ctk.CTkLabel(frame, text=label_text)
    label.pack(side="left", padx=5)
    
    slider = ctk.CTkSlider(frame, from_=from_, to=to_, variable=variable, number_of_steps=(to_ - from_))
    slider.pack(side="right", fill="x", expand=True)
    return slider

create_label_slider("Quality:", quality, 1, 10)
create_label_slider("Last Episode:", lastep, 1, 100)

# Checkbox para IDM
idm_frame = ctk.CTkFrame(app)
idm_frame.pack(pady=5, padx=10, fill="x")

idm_label = ctk.CTkLabel(idm_frame, text="Use IDM:")
idm_label.pack(side="left", padx=5)

idm_checkbox = ctk.CTkCheckBox(idm_frame, text="", variable=idm)
idm_checkbox.pack(side="right")

# Función para guardar los valores y mostrar un mensaje
def save_values():
    print("login_url:", login_url.get())
    print("base_url:", base_url.get())
    print("serie:", serie.get())
    print("quality:", quality.get())
    print("lastep:", lastep.get())
    print("idm:", idm.get())

    ctk.CTkMessageBox.show_info("Información", "Variables guardadas exitosamente.")

# Botón para guardar
save_button = ctk.CTkButton(app, text="Guardar", command=save_values)
save_button.pack(pady=20)

# Ejecutar la aplicación
app.mainloop()


In [2]:
episodios_descargas = {}

def transformar_url(url_base, numero_episodio):
    url_modificada = re.sub(r"/category", "", url_base)
    episodio_url = f"{url_modificada}-episode-{numero_episodio}"
    return episodio_url

session = requests.Session()
login_page = session.get(login_url)
soup = bs(login_page.text, 'html.parser')
csrf_token = soup.find('input', {'name': '_csrf'})['value']

login_data = {
    "email": email,
    "password": password,
    "_csrf": csrf_token,
    "remember": "1"
}

login_response = session.post(login_url, data=login_data)


In [3]:
if (idm):
    if "Logout" in login_response.text:
        print("Login exitoso!")
        
        def obtener_links_descarga(session, url):
            response = session.get(url)
            
            if response.status_code == 200:
                soup = bs(response.content, 'html.parser')
                list_download_div = soup.find('div', class_='list_dowload')
                
                if list_download_div:
                    cf_download_div = list_download_div.find('div', class_='cf-download')
                    
                    if cf_download_div:
                        links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                        if 1 <= quality <= len(links):
                            return links[quality - 1]
                        else:
                            print("La calidad seleccionada está fuera de rango.")
                            return None
                    else:
                        print("No se encontró el div 'cf-download'.")
                else:
                    print("No se encontró el div 'list_dowload'.")
            else:
                print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            
            return None

        for i in range(1, lastep + 1):
            episodio_url = transformar_url(base_url, i)
            link_descarga = obtener_links_descarga(session, episodio_url)
            
            if link_descarga:
                episodios_descargas[episodio_url] = link_descarga
                print(f"Episodio {i}: {link_descarga}")
            else:
                episodios_descargas[episodio_url] = None
                print(f"No se encontró un link de descarga para el episodio {i}")
        
        print("\nDiccionario de episodios con sus links de descarga:")
        print(episodios_descargas)
        
        def descargar_con_idm(episodios_dict):
            idm_path = r"C:\Program Files (x86)\Internet Download Manager\IDMan.exe"  # Ruta de IDM, ajusta si es necesario
            if not os.path.exists(idm_path):
                print("IDM no se encontró en la ruta especificada.")
                return
            
            for episodio, url_descarga in episodios_dict.items():
                if url_descarga:
                    numero_episodio = re.search(r'-episode-(\d+)', episodio).group(1)
                    file_name = f"${serie}_Episode_{numero_episodio}.mp4"
                    file_full_path = os.path.join(file_path, file_name)
                    
                    print(f"Descargando {file_name} desde {url_descarga} ...")
                    
                    subprocess.run([idm_path, "/d", url_descarga, "/p", file_path, "/f", file_name, "/n", "/a"])
                    
            # Comienza todas las descargas agregadas a la cola
            subprocess.run([idm_path, "/s"])
            print("Todas las descargas se han añadido a IDM.")

        descargar_con_idm(episodios_descargas)

    else:
        print("Error en el login")

else:
    if "Logout" in login_response.text:
        print("Login exitoso!")
        
        def obtener_links_descarga(session, url):
            response = session.get(url)
            
            if response.status_code == 200:
                soup = bs(response.content, 'html.parser')
                list_download_div = soup.find('div', class_='list_dowload')
                
                if list_download_div:
                    cf_download_div = list_download_div.find('div', class_='cf-download')
                    
                    if cf_download_div:
                        links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                        if 1 <= quality <= len(links):
                            return links[quality - 1]
                        else:
                            print("La calidad seleccionada está fuera de rango.")
                            return None
                    else:
                        print("No se encontró el div 'cf-download'.")
                else:
                    print("No se encontró el div 'list_dowload'.")
            else:
                print(f"Error al acceder a la página. Código de estado: {response.status_code}")
            
            return None

        for i in range(1, lastep + 1):
            episodio_url = transformar_url(base_url, i)
            link_descarga = obtener_links_descarga(session, episodio_url)
            
            if link_descarga:
                episodios_descargas[episodio_url] = link_descarga
                print(f"Episodio {i}: {link_descarga}")
            else:
                episodios_descargas[episodio_url] = None
                print(f"No se encontró un link de descarga para el episodio {i}")
        
        print("\nDiccionario de episodios con sus links de descarga:")
        print(episodios_descargas)
        
        def descargar_episodios(episodios_dict):
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            
            for episodio, url_descarga in episodios_dict.items():
                if url_descarga:
                    numero_episodio = re.search(r'-episode-(\d+)', episodio).group(1)
                    file_name = f"OnePiece_Episode_{numero_episodio}.mp4"
                    file_full_path = os.path.join(file_path, file_name)
                    
                    print(f"Descargando {file_name} desde {url_descarga} ...")
                    
                    with session.get(url_descarga, stream=True) as response:
                        response.raise_for_status()
                        with open(file_full_path, 'wb') as file:
                            for chunk in response.iter_content(chunk_size=8192):
                                file.write(chunk)
                    
                    print(f"Descarga de {file_name} completada.")
        
        descargar_episodios(episodios_descargas)

    else:
        print("Error en el login")

Login exitoso!
Episodio 1: https://ggredi.info/download.php?url=aHR0cHM6LyURASDGHUSRFSJGYfdsffsderFStewthsfSFtrfteAawehyfcghysfdsDGDYdgdsfsdfwstdgdsgtertsdf82eGM1cHBlOWVpLmFuZjU5OC5jb20vdXNlcjEzNDIvYzc1MWJhYjE5MzlhMmI4MzAyMDU2NWUxYWMyNDI4OTYvRVAuMS52MS4xMDgwcC5tcDQ/dG9rZW49NFdKZEs3Z0J0T0hxdkp1djE1dDVDUSZleHBpcmVzPTE3Mjc3NzQyMjAmaWQ9MzUxOCZ0aXRsZT0oMTkyMHgxMDgwLWdvZ29hbmltZSlvbmUtcGllY2UtZXBpc29kZS0xLm1wNA==
Episodio 2: https://ggredi.info/download.php?url=aHR0cHM6LyAdrefsdsdfwerFrefdsfrersfdsrfer363435349URASDGHUSRFSJGYfdsffsderFStewthsfSFtrftesdfseWVpYnU0bmM3LmFuZjU5OC5jb20vdXNlcjEzNDIvYzc1MWJhYjE5MzlhMmI4MzAyMDU2NWUxYWMyNDI4OTYvRVAuMi52MS4xMDgwcC5tcDQ/dG9rZW49Q1YzZ3V2RERZc2IwdXE5V3RhNERqQSZleHBpcmVzPTE3Mjc3NzQyMjEmaWQ9MzUyMSZ0aXRsZT0oMTkyMHgxMDgwLWdvZ29hbmltZSlvbmUtcGllY2UtZXBpc29kZS0yLm1wNA==
Episodio 3: https://ggredi.info/download.php?url=aHR0cHM6LyAawehyfcghysfdsDGDYdgdsfsdfwstdgdsgtert9URASDGHUSRFSJGYfdsffsderFStewthsfSFtrftesdfjc2lhaTRua3FnLmFuZjU5OC5jb20vdXNlcjEzNDIvYzc1MWJhYj